In [97]:
from scipy.sparse import csr_matrix, lil_matrix
from scipy.io import mmread
import numpy as np
import pandas as pd
import utils
import scores
from math import pi, floor, ceil
from random import randint
from contai

In [4]:
data, adjacency, edges = utils.loadData("small")

In [5]:
solution = data["assignment"]

In [4]:
solution_perimeter = scores.perimeter(data, adjacency, edges, solution)

In [8]:
area = data["area"].groupby(solution).sum()

In [10]:
4*pi*area/(solution_perimeter**2)

assignment
1    0.268645
2    0.248994
3    0.231095
dtype: float64

In [9]:
area/solution_perimeter

assignment
1    519.024053
2    713.145716
3    809.003866
dtype: float64

In [18]:
np.unique(solution)

array([1, 2, 3])

In [20]:
pd.Series(np.zeros(len(np.unique(solution))), index=np.unique(solution))

1    0.0
2    0.0
3    0.0
dtype: float64

In [13]:
area

assignment
1    1.260104e+07
2    2.566717e+07
3    3.558940e+07
Name: area, dtype: float64

# Initializer

In [100]:
data, adjacency, edges = utils.loadData("small")

In [167]:
# How many total units to assign
n = data.shape[0]
# How many districts to build
k = 3
# Solution array (initially nothing is assigned)
sol = np.zeros(n, dtype="int32")
# Select k seed zones as starting elements
seeds = data[data["outer_edge"] > 0].sample(k).index.values

# Set assignment for each of the seeds
for i, v in enumerate(seeds):
    sol[v] = i+1

In [168]:
pool = seeds.tolist()

In [169]:
# While there are still elements in pool
while pool:
    # Select an index available in pool
    i = randint(0, len(pool)-1)
    # Pop that item out
    u = pool.pop(i)
    # Get list of zones adjacent to u
    for v in adjacency.rows[u]:
        if sol[v] == 0 and v not in pool:
            pool.append(v)
            sol[v] = sol[u]

In [170]:
print(", ".join(map(str,sol)))

3, 3, 2, 1, 2, 2, 2, 2, 1, 3, 3, 1


In [109]:
np.nonzero(sol == 3)[0]

array([5, 6])

# Mutation

## Shift

"Shift moves a number of units from one district to a neighboring district. To ensure that a shift does not violate contiguity, the selected units include at least one unit on the boundary of the sending and receiving district." (PEAR Algorithm 3)

In [269]:
# subzone is a list of units that are all contiguous
# returns a list of units that are adjacent to subzone
# TODO: make sure they are in the right zone
def subzone_neighbors(ind, subzone, zid):
    neighbors = set()
    for r in adjacency.rows[subzone]:
        neighbors = neighbors | set(r)
    
    return [i for i in (neighbors - set(subzone)) if ind[i] == zid]

In [271]:
sz = subzone_neighbors(sol, np.nonzero(sol == 3)[0], 3)
print(sz)

[]


In [231]:
def edge_units(ind, src, dst):
    # Set of all units found
    units = set()
    
    src_zone = np.nonzero(ind == src)[0]
    dst_zone = np.nonzero(ind == dst)[0]
    
    # For each unit u in the source zone...
    for u in src_zone:
        # For each unit v adjacent to u...
        for v in adjacency.rows[u]:
            # If v is in the destination zone...
            if v in dst_zone:
                # Then u is an edge unit
                units = units | set([u])
                # Don't need to look at any other units adaject to u if it's already confirmed an edge unit
                break
    return list(units)
    

In [232]:
print(edge_units(sol, 3, 2))

[0, 9]


### Check Contiguity

Algorith 1 from PEAR. Given a solution (individual), and a group of blocks to move from one zone to another (subzone), make sure that removing those from their current zone doesn't break contiguity. 

In [321]:
from collections import defaultdict
def bfs(G, origin):
    visited = defaultdict(lambda: False)
    queue = []
    connected = []
    
    queue.append(origin)
    visited[origin] = True
    
    while queue:
        s = queue.pop()
        connected.append(s)
        
        for i in G[s]:
            if visited[i] == False:
                queue.append(i)
                visited[i] = True
                
    return connected
    

In [362]:
def contiguity_check(ind, subzone, src):
    # If no subzones are being removed then contiguity is not affected
    if len(subzone) == 0:
        return True
    # check that at least one unit of subzone is on a zone boundary
    # (not really necessary because we control the generative process, guaranteeing this)
    # V = {} - units that are neighbors of the subzone, and in the same zone as them.
    neighbors = subzone_neighbors(ind, subzone, src)
    
    # G = {}; connectivity graph as adjacency list
    G = {}
    for u in neighbors:
        G[u] = []
        for v in adjacency.rows[u]:
            if v in neighbors:
                G[u].append(v)
        
    c = len(bfs(G, list(G.keys())[0]))
    return c > 0 and c == len(neighbors)




In [363]:
cc_ind = [2, 2, 1, 2, 2, 1, 3, 3, 1, 3, 3, 1]
contiguity_check(cc_ind, [1, 3], 2)

assert not contiguity_check(cc_ind, [0, 3], 2)
assert not contiguity_check(cc_ind, [9], 3)
assert contiguity_check(cc_ind, [11], 3)
assert contiguity_check(cc_ind, [1, 3], 2)



In [356]:
# Define externally, number (15) taken from PEAR
max_mutation_units = 2

# src and dst are adjacent zones
def shift(ind, src, dst):
    # Randomly select up to two adjacent units in src bordering on dst
    # TODO: current code only selects one
    eu = edge_units(ind, src, dst)
    if len(eu) == 0:
        return ind
    
    subzone = [np.random.choice(edge_units(ind, src, dst))]
    
    # while size of subzone < max mutation units
    while len(subzone) < max_mutation_units:
        # Get list of neighbor units (U) of subzone
        neighbors = subzone_neighbors(ind, subzone, src)
        if len(neighbors) > 0:
            # Get random number q in 1:|U|
            q = randint(1, len(neighbors))
            # Randomly choose subset of U with |U'| = q
            subset = np.random.choice(neighbors, q, replace=False )
            # subzone = subzone union U'
            subzone = list(set(subzone) | set(subset))
            
    if contiguity_check(ind, subzone, src):
        # dst = dst union subzone
        # src = src - subzone
        # aka reassign subzone to dst id
        for i in subzone:
            ind[i] = dst
        
    return ind

In [ ]:
tmp_ind = np.array([2, 2, 1, 2, 2, 1, 3, 3, 1, 3, 3, 1])

In [360]:
#src = np.nonzero(sol == 3)[0]
#dst = np.nonzero(sol == 2)[0]
src = 1
dst = 2
tmp_ind = shift(tmp_ind, src, dst)
print(tmp_ind)


[2 1 1 1 2 1 2 2 1 3 3 1]
[2, 5]
[8]
[2 1 2 1 2 2 2 2 1 3 3 1]


## Mutate

"Mutate makes a sequence of shifts to balance metrics such as population deviation. This sequence may have one or more cyclic shifts."

In [81]:
# Start with a current solution; using sol from above
# Need a shuffled list of possible zones to shift
# Paper recommends Fisher-Yates, which is what numpy uses
zones = [i for i in range(1, k+1)]
np.random.shuffle(zones)



In [98]:
# NEED: the population information
pop_threshold = 0.05
total_pop = data["block_pop_total"].sum()
min_pop = floor(total_pop*(1-pop_threshold))
max_pop = ceil(total_pop*(1+pop_threshold))

In [ ]:
# for each z in zones
#   if z is a source zone of previous shifts
#     continue

In [127]:
data

,Unnamed: 0,GEOID,area,perimeter,outer_edge,assignment
0,4,53053073122,5.032197e+06,11813.664235,0.000000,1
1,53,53053073126,1.403617e+06,5014.567928,0.000000,1
2,551,53053071309,4.133641e+06,13086.810210,7367.835131,2
3,572,53053073121,2.848713e+06,7147.099962,0.000000,1
4,586,53053073120,3.316514e+06,9756.762195,0.000000,1
5,831,53053071304,5.373849e+06,11423.230843,5320.001284,2
6,832,53053071208,3.689688e+06,8980.737923,6201.710903,3
7,890,53053073111,3.732590e+06,9662.924189,2021.050246,3
8,892,53053073108,9.902125e+06,14729.911244,3405.502371,2
9,1160,53053073123,9.665775e+06,20030.644348,6273.267507,3


In [238]:
solution[9]

3

In [257]:
sol

array([3, 3, 2, 1, 2, 2, 2, 2, 1, 3, 3, 1], dtype=int32)